In [ ]:
from scipy._lib._test_ccallback import test_get_data_capsule

from models.graph_transformers.SAT.sat import models
from models.graph_transformers.SAT.sat import layers
import torch
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader


def ptr_to_complete_edge_index(ptr):
    from_lists = [torch.arange(ptr[i], ptr[i + 1]).repeat_interleave(ptr[i + 1] - ptr[i]) for i in range(len(ptr) - 1)]
    to_lists = [torch.arange(ptr[i], ptr[i + 1]).repeat(ptr[i + 1] - ptr[i]) for i in range(len(ptr) - 1)]
    combined_complete_edge_index = torch.vstack((torch.cat(from_lists, dim=0), torch.cat(to_lists, dim=0)))
    return combined_complete_edge_index
test_data = torch.FloatTensor([1., 2., 3., 4., 5.]).unsqueeze(1)
edge_index = torch.LongTensor([[0,1,2],[1, 2, 3]])

test_graph = Data(x=test_data, edge_index=edge_index)
test_data2 = torch.FloatTensor([-1., -2., -3., -4., 10]).unsqueeze(1)
edge_index2 = torch.LongTensor([[0,1],[1, 2]])
test_graph2 = Data(x=test_data2, edge_index=edge_index2)
loader = iter(DataLoader([test_graph, test_graph2], batch_size=2))
batch = next(loader)


batch.complete_edge_index = torch.LongTensor([[0,1,5],[1,2,6]])
# batch.complete_edge_index = ptr_to_complete_edge_index(batch.ptr)


model = models.GraphTransformer(in_size=1,
                                num_class=1,
                                in_embed=False,
                                d_model=3,
                                dim_feedforward=2,
                                num_heads=1,
                                num_layers=1,
                                abs_pe=False,
                                abs_pe_dim=None,

                                use_edge_attr=False,
                                num_edge_features=0,
                                dropout=0.,
                                k_hop=2)


#model(batch)

In [ ]:
amrlayer = layers.AttentionAMR(1, 0, 1, 0, True).cuda()

In [ ]:
batch.ptr

In [ ]:
edge_index_source = torch.LongTensor([[i for i in torch.arange(batch.edge_index.shape[1])], [batch.edge_index[0][i] for i in torch.arange(batch.edge_index.shape[1])]]).cuda()
edge_index_target = torch.LongTensor([[i for i in torch.arange(batch.edge_index.shape[1])], [batch.edge_index[1][i] for i in torch.arange(batch.edge_index.shape[1])]]).cuda()

amrlayer(batch.x.cuda(), batch.x.cuda(), batch.edge_index.cuda(), edge_index_source, edge_index_target, ptr=batch.ptr.cuda())

In [ ]:
embed = torch.nn.Linear(1000, 128)

In [ ]:
import pickle
with open("data/hol4/data/torch_graph_dict.pk", "rb") as f:
    tg = pickle.load(f)
test_graph_1 = tg[list(tg.keys())[0]]
test_graph_1
test_graph_2 = tg[list(tg.keys())[2]]
test_graph_2
loader = iter(DataLoader([test_graph_1, test_graph_2], batch_size=2))
batch = next(loader)

In [ ]:

from models.graph_transformers.SAT.sat import layers
amrlayer = layers.AttentionAMR(128,0,2,0,True)

x = embed(batch.x)

out_s, out_t = amrlayer(x, x, batch.edge_index)

In [ ]:
amrlayer(out_s, out_t, batch.edge_index)

In [ ]:
batch.edge_index

In [ ]:

batch.edge_attr = batch.edge_attr.long()
model = models.AMRTransformer(in_size=1000,num_class=0,d_model=128,num_heads=4,dim_feedforward=128,dropout=0.1,use_edge_attr=True,num_layers=1,layer_norm=True,in_embed=False).cuda()
model(batch.cuda())

In [ ]:
torch.tensor([1.,2.,3.,4.,5.]).to(torch.tensor([1.,1.,2.,5,5234,643,2345]))

In [ ]:
batch.x